In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN_Wmultitask_32_opp"

## Importing Libraries

In [4]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [6]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [7]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [8]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [9]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [10]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


variables_weights = np.ones((n_classes,))

In [11]:
pos_weight.shape

(19,)

In [12]:
pos_weight

array([  0.28595522,  42.44204322,  42.96023857,  45.06666667,
        44.49794239,  34.32268371,  35.42833608,  50.54312354,
        54.97974684,  62.17714286,  69.87179487,  66.41463415,
        71.02605863,  57.96533333,  57.4973545 ,  39.35036496,
        12.09177028,  47.81236203,   4.77487595])

In [13]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [14]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [15]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [16]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = True,
                     trainable_last = True,
                     variables_weights = variables_weights)

In [ ]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [ ]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.066 vs. 0.068
F1 = 0.103 vs. 0.101
Accuracy = 0.005 vs. 0.056
CHANGE DETECTION
AUC = 0.173 vs. 0.177
F1 = 0.290 vs. 0.294
Precision = 0.173 vs. 0.175
Recall = 0.883 vs. 0.935
Validation
ACTIVITY RECOGNITION
AUC = 0.065 vs. 0.063
F1 = 0.100 vs. 0.092
Accuracy = 0.003 vs. 0.056
CHANGE DETECTION
AUC = 0.119 vs. 0.098
F1 = 0.216 vs. 0.182
Precision = 0.135 vs. 0.100
Recall = 0.538 vs. 0.995
Minibatch Loss= 1.308718
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.519 vs. 0.066
F1 = 0.560 vs. 0.096
Accuracy = 0.645 vs. 0.056
CHANGE DETECTION
AUC = 0.344 vs. 0.153
F1 = 0.451 vs. 0.269
Precision = 0.335 vs. 0.156
Recall = 0.690 vs. 0.962
Validation
ACTIVITY RECOGNITION
AUC = 0.339 vs. 0.062
F1 = 0.424 vs. 0.091
Accuracy = 0.533 vs. 0.056
CHANGE DETECTION
AUC = 0.295 vs. 0.098
F1 = 0.393 vs. 0.182
Precision = 0.313 vs. 0.100
Recall = 0.530 vs. 0.989
Minibatch Loss= 0.382019
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.608 vs. 0.068


In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/CNN_activity_multitask_64_opp/CNN_activity_multitask_64_opp-20000')


In [ ]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

In [ ]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [ ]:
1